In [18]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [49]:
df = pd.read_csv("DFF_result.csv")

# Clean data.

# Pulse width values for Pass cases & Current percentage values for Fail cases. 
df.loc[df['fail'] == 'pass', 'Junction Values'] = df.loc[df['fail'] == 'pass', 'Pulse width info']
df.loc[df['fail'] == 'fail', 'Junction Values'] = df.loc[df['fail'] == 'fail', 'Current percentage']


# One-hot encode pass/fail.
drop_col = ['cus', 'real_flux', 'Faulty Junction', 'Pulse magnitude info', 'Phase state info', 'Pulse width info', 'Current percentage' ]
df = df.drop(drop_col, axis=1)

# df.head()

In [50]:
# Rule 1: Initial Bias is between 60%-80%. 
# Rule 2: Junction 4 bias <55%. 


# Create rule 1, rule 2, pass/fail truth table.
for val in df['Junction Values']:
    val = eval(str(val))
    if 0.6 <= (val['B01RX1']) <= 0.8:
        df['Rule 1'] = 1
    else:
        df['Rule 1'] = 0
    if val['B4'] <0.55:
        df['Rule 2'] = 1
    else:
        df['Rule 2'] = 0

columns = df.columns.tolist()
columns.append(columns.pop(3))
df["fail"] = df["fail"].replace({"pass": 1, "fail": 0})
df = df[columns]

# df.head()

In [ ]:
# Create buckets.
condition0 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 0, 0]).all(axis=1)
condition0_df = df[condition0]
print(len(condition0_df))

condition1 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 0, 1]).all(axis=1)
condition1_df = df[condition1]
print(len(condition1_df))

condition2 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 1, 0]).all(axis=1)
condition2_df = df[condition2]
print(len(condition2_df))

condition3 = (df[['Rule 1', 'Rule 2', 'fail']] == [0, 1, 1]).all(axis=1)
condition3_df = df[condition3]
print(len(condition3_df))

condition4 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 0, 0]).all(axis=1)
condition4_df = df[condition4]
print(len(condition4_df))

condition5 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 0, 1]).all(axis=1)
condition5_df = df[condition5]
print(len(condition5_df))

condition6 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 1, 0]).all(axis=1)
condition6_df = df[condition6]
print(len(condition6_df))

condition7 = (df[['Rule 1', 'Rule 2', 'fail']] == [1, 1, 1]).all(axis=1)
condition7_df = df[condition7]
print(len(condition7_df))

In [ ]:
condition6_df['driver'] = condition6_df['config'].str.split('_').str[0]
condition6_df['load'] = condition6_df['config'].str.split('_').str[6]
condition6_df['splitterNum'] = condition6_df['config'].str.split('_').str[2]

In [54]:
condition6_df = pd.get_dummies(condition6_df, columns = ["driver", "load", "splitterNum"])
# condition6_df

In [84]:
import Quine_McCluskey_Functions as qm
truthTable_6 = condition6_df.iloc[:, 9:]
truthTable_6

,driver_and,driver_dff,driver_inv,driver_or,driver_xor,load_and,load_dff,load_inv,load_or,load_xor,splitterNum_0,splitterNum_1,splitterNum_2,splitterNum_3
1,1,0,0,0,0,1,0,0,0,0,1,0,0,0
12,1,0,0,0,0,1,0,0,0,0,1,0,0,0
16,1,0,0,0,0,1,0,0,0,0,1,0,0,0
17,1,0,0,0,0,1,0,0,0,0,1,0,0,0
19,1,0,0,0,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39996,0,0,0,0,1,0,0,0,0,1,0,0,0,1
39997,0,0,0,0,1,0,0,0,0,1,0,0,0,1
39998,0,0,0,0,1,0,0,0,0,1,0,0,0,1
39999,0,0,0,0,1,0,0,0,0,1,0,0,0,1


In [ ]:
minterms_6 = []
for row in truthTable_6.values:
    binary = "".join(map(str, row))
    minterms_6.append(binary)

minterms_6 = list(set(minterms_6))
minterms_6.sort()

size = 12
groups,all_pi = {},set()

In [94]:
# Primary grouping starts
for minterm in minterms_6:
    try:
        groups[bin(int(minterm)).count('1')].append(bin(int(minterm))[2:].zfill(size))
    except KeyError:
        groups[bin(int(minterm)).count('1')] = [bin(int(minterm))[2:].zfill(size)]
# Primary grouping ends

#Primary group printing starts
print("\n\n\n\nGroup No.\tminterms_6\tBinary of minterms_6\n%s"%('='*50))
for i in sorted(groups.keys()):
    print("%5d:"%i) # Prints group number
    for j in groups[i]:
        print("\t\t    %-20d%s"%(int(j,2),j)) # Prints minterm and its binary representation
    print('-'*50)
#Primary group printing ends

# Process for creating tables and finding prime implicants starts
while True:
    tmp = groups.copy()
    groups,m,marked,should_stop = {},0,set(),True
    l = sorted(list(tmp.keys()))
    for i in range(len(l)-1):
        for j in tmp[l[i]]: # Loop which iterates through current group elements
            for k in tmp[l[i+1]]: # Loop which iterates through next group elements
                res = qm.compare(j,k) # Compare the minterms_6
                if res[0]: # If the minterms_6 differ by 1 bit only
                    try:
                        groups[m].append(j[:res[1]]+'-'+j[res[1]+1:]) if j[:res[1]]+'-'+j[res[1]+1:] not in groups[m] else None # Put a '-' in the changing bit and add it to corresponding group
                    except KeyError:
                        groups[m] = [j[:res[1]]+'-'+j[res[1]+1:]] # If the group doesn't exist, create the group at first and then put a '-' in the changing bit and add it to the newly created group
                    should_stop = False
                    marked.add(j) # Mark element j
                    marked.add(k) # Mark element k
        m += 1
    local_unmarked = set(qm.flatten(tmp)).difference(marked) # Unmarked elements of each table
    all_pi = all_pi.union(local_unmarked) # Adding Prime Implicants to global list
    print("Unmarked elements(Prime Implicants) of this table:",None if len(local_unmarked)==0 else ', '.join(local_unmarked)) # Printing Prime Implicants of current table
    if should_stop: # If the minterms_6 cannot be combined further
        print("\n\nAll Prime Implicants: ",None if len(all_pi)==0 else ', '.join(all_pi)) # Print all prime implicants
        break
    # Printing of all the next groups starts
    print("\n\n\n\nGroup No.\tminterms_6\tBinary of minterms_6\n%s"%('='*50))
    for i in sorted(groups.keys()):
        print("%5d:"%i) # Prints group number
        for j in groups[i]:
            print("\t\t%-24s%s"%(','.join(qm.findminterms(j)),j)) # Prints minterms_6 and its binary representation
        print('-'*50)
    # Printing of all the next groups ends
# Process for creating tables and finding prime implicants ends

# Printing and processing of Prime Implicant chart starts
sz = len(str(minterms_6[-1])) # The number of digits of the largest minterm
chart = {}
print('\n\n\nPrime Implicants chart:\n\n    minterms_6    |%s\n%s'%(' '.join((' '*(sz-len(str(i))))+str(i) for i in minterms_6),'='*(len(minterms_6)*(sz+1)+16)))
for i in all_pi:
    merged_minterms_6,y = qm.findminterms(i),0
    print("%-16s|"%','.join(merged_minterms_6),end='')
    
# Printing and processing of Prime Implicant chart ends

EPI = qm.findEPI(chart) # Finding essential prime implicants
print("\nEssential Prime Implicants: "+', '.join(str(i) for i in EPI))
qm.removeTerms(chart,EPI) # Remove EPI related columns from chart

if(len(chart) == 0): # If no minterms_6 remain after removing EPI related columns
    final_result = [qm.findVariables(i) for i in EPI] # Final result with only EPIs
else: # Else follow Petrick's method for further simplification
    P = [[qm.findVariables(j) for j in chart[i]] for i in chart]
    while len(P)>1: # Keep multiplying until we get the SOP form of P
        P[1] = qm.multiply(P[0],P[1])
        P.pop(0)
    final_result = [min(P[0],key=len)] # Choosing the term with minimum variables from P
    final_result.extend(qm.findVariables(i) for i in EPI) # Adding the EPIs to final solution
print('\n\nSolution: F = '+' + '.join(''.join(i) for i in final_result))





Group No.	minterms_6	Binary of minterms_6
   10:
		    1100000001          1000001100100001010101100000001
--------------------------------------------------
   11:
		    1100000010          1000001100100001010101100001010
		    10000010001         1001010100000011000000101100010001
		    10100000001         1001011010000000011100010100000001
--------------------------------------------------
   12:
		    1001001000          111011101010100001000000101000
		    1010000001          111100001100110110000010000001
		    1100000100          1000001100100001010101101100100
		    10000010010         1001010100000011000000101100011010
		    10100000010         1001011010000000011100010100001010
--------------------------------------------------
   13:
		    1000100100          111011100111000101000100000100
		    1001000001          111011101010100000110001000001
		    1010000010          111100001100110110000010001010
		    1100001000          1000001100100001010111011101000
		    100000